In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

In [2]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [4]:
#encoding variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [5]:
##One-hot encode geography column
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
##combine it with original dataframe and then dropping original one
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
##splitting dependent and independent variables or features and target
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [8]:
## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [10]:
#save encoders and scaler for later use
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANNRegression problem statement

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)#output for regression-->linear activation function
])
##compile the model
model.compile(
    optimizer='adam',
    loss='mean_absolute_error',
    metrics=['mae']
)

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                832       
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [22]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [23]:
### Train the model
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 1s 3ms/step - loss: 100375.0156 - mae: 100375.0156 - val_loss: 98507.3750 - val_mae: 98507.3750
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 99593.8516 - mae: 99593.8516 - val_loss: 96933.2734 - val_mae: 96933.2734
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 96863.3438 - mae: 96863.3438 - val_loss: 92931.3047 - val_mae: 92931.3047
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 91506.2812 - mae: 91506.2812 - val_loss: 86280.4844 - val_mae: 86280.4844
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 83715.4688 - mae: 83715.4688 - val_loss: 77652.6953 - val_mae: 77652.6953
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 74550.8672 - mae: 74550.8672 - val_loss: 68723.3828 - val_mae: 68723.3828
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 65692.2031 - mae: 65692.203

In [24]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6007 (pid 19656), started 0:00:07 ago. (Use '!kill 19656' to kill it.)

In [28]:
##evaluate the model
test_loss,test_mae=model.evaluate(X_test,y_test)
print("test_mae",test_mae)

63/63 [==============================] - 0s 1ms/step - loss: 50304.3984 - mae: 50304.3984
test_mae 50304.3984375


In [29]:
model.save('regression_model.h5')

c:\Users\vamsi\Documents\Udemy\restart\Deep Learning\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
